Antes de iniciar, certifique-se que obteve uma maquina com GPU ou TPU




Seguir os passos descritos aqui https://www.kaggle.com/general/74235




In [1]:
# ! pip install -q kaggle
# from google.colab import files
# files.upload() #aqui faz o upload do kaggle.json que voce gerou. Ver link acima.

Baixar e extrair os dados

In [2]:
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d kmader/food41
# !unzip food41.zip

In [3]:
# import os
# os.remove('food41.zip')

Separar em treino e teste de acordo com a sugestao do autor

In [4]:
import os
with open('dataset/meta/meta/train.txt') as trainfile:
    trainlines = trainfile.readlines()
    trainlines = [line.rstrip()+'.jpg' for line in trainlines]

with open('dataset/meta/meta/test.txt') as testfile:
    testlines = testfile.readlines()
    testlines = [line.rstrip()+'.jpg' for line in testlines]

# !mkdir train/
# !mkdir test/

#create directories
with open('dataset/meta/meta/classes.txt') as classfile:
    classlines = classfile.readlines()
    classlines = [line.rstrip() for line in classlines]

for class_ in classlines:
    try:
        os.makedirs('dataset/train/'+class_)
    except:
        pass
    try:
        os.makedirs('dataset/test/'+class_)
    except:
        pass

#move files to corresponding folder
for tl in trainlines:
    try:
        os.replace('dataset/images/'+tl, 'dataset/train/'+tl)
    except:
        pass
for tl in testlines:
    try:
        os.replace('dataset/images/'+tl, 'dataset/test/'+tl)
    except:
        pass

Limpa o que nao vai ser usado

In [5]:
#clean up
for class_ in classlines:
    try:
        os.rmdir('dataset/images/'+class_)
    except:
        pass
try:
    os.rmdir('dataset/images/')
except:
    pass

Aqui comeca de fato o modelo. O codigo abaixo e apenas uma sugestao.

In [6]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.models import Model
from keras.optimizers import Adam

In [7]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

datagen_test = ImageDataGenerator(
    rescale=1./255
    #insert only preprocessing functions
)

In [8]:
train_batches = datagen.flow_from_directory('dataset/train/', batch_size=64, class_mode = 'categorical', target_size=(224, 224))
test_batches = datagen_test.flow_from_directory('dataset/test/', batch_size=64, class_mode = 'categorical', target_size=(224, 224))

Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


Esta etapa pode gerar um problema. Se o flow from directory detectar mais de 101 classes, possivelmente existem arquivos temporarios do notebook escondidos dentro da pasta. Neste caso remover com !rm -r (train ou test)/.ipynb_checkpoints/

In [9]:
import scipy

print(train_batches)
print(len(train_batches))
for each in train_batches:
    print(each[0].shape)
    print(each[1].shape)
    break

1184
(64, 224, 224, 3)
(64, 101)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Activation

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=train_batches[0][0].shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_batches.class_indices), activation='softmax'))
# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# Display the model summary
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 activation (Activation)     (None, 224, 224, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 222, 222, 32)      9248      
                                                                 
 activation_1 (Activation)   (None, 222, 222, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 111, 111, 32)      0         
                                                        

C:\Users\Marli\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
import tensorflow.compat.v1 as tf 

tf.enable_eager_execution(tf.ConfigProto(log_device_placement=True)) 

print(tf.add([1.0, 2.0], [3.0, 4.0])) 

tf.Tensor([4. 6.], shape=(2,), dtype=float32)


In [12]:
import tensorflow as tf
# Fit the model2 to the data with augmented training data
with tf.device('/GPU:0'):
    history = model.fit(
        train_batches,
        steps_per_epoch = 75750/64,
        epochs=10,
        validation_data = test_batches, #create a validation set if needed
        validation_steps= 25250/64,
        # use_multiprocessing=True,
        workers=8,
        max_queue_size=512,
    )

Epoch 1/10
1183/1183 [==============================] - 1684s 1s/step - loss: 4.5039 - accuracy: 0.0292 - val_loss: 4.3883 - val_accuracy: 0.0704
Epoch 2/10
1183/1183 [==============================] - 1691s 1s/step - loss: 4.2963 - accuracy: 0.0559 - val_loss: 4.0445 - val_accuracy: 0.0986
Epoch 3/10
1183/1183 [==============================] - 2792s 2s/step - loss: 4.1708 - accuracy: 0.0731 - val_loss: 3.9476 - val_accuracy: 0.1086
Epoch 4/10
1183/1183 [==============================] - 2755s 2s/step - loss: 4.0803 - accuracy: 0.0872 - val_loss: 3.7917 - val_accuracy: 0.1380
Epoch 5/10
1183/1183 [==============================] - 1837s 2s/step - loss: 4.0064 - accuracy: 0.0977 - val_loss: 3.8044 - val_accuracy: 0.1315
Epoch 6/10
1183/1183 [==============================] - 2821s 2s/step - loss: 3.9415 - accuracy: 0.1078 - val_loss: 3.7132 - val_accuracy: 0.1437
Epoch 7/10
1183/1183 [==============================] - 2280s 2s/step - loss: 3.8964 - accuracy: 0.1164 - val_loss: 3.6279 -

Daqui para baixo, podem ser gerados os resultados. A variavel hist pode ser usada para plotar as curvas. Para obter as predicoes, usar model.predict(test_batches). Matrizes de confusao e metricas podem ser facilmente calculadas usando o pacote scikit-learn.

In [13]:
test_loss, test_accuracy = model.evaluate(test_batches)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

395/395 [==============================] - 47s 119ms/step - loss: 3.5566 - accuracy: 0.1692
Test Accuracy: 16.92%
